## RAG: Retrieval Augmented Generation.

- Large language models (LLMs) have a limited context size.
- TLDR
- Not all context is relevant to a given question
- Query -> Search -> Results -> (LLM) -> Answer

## Keyword VS Semantic Search 
![Vector](https://blog.dataiku.com/hs-fs/hubfs/dftt%202.webp?width=1346&height=632&name=dftt%202.webp)

from https://blog.dataiku.com/semantic-search-an-overlooked-nlp-superpower

![Emb_search](figures/emb_search.png)

from https://sreent.medium.com/llms-embeddings-and-vector-search-d4bd9362df56

In [1]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25
! pip3 install -qU langchain_chroma

In [67]:

%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [68]:
import warnings

warnings.filterwarnings("ignore")

In [69]:
from langchain_upstage import UpstageEmbeddings

embeddings_model = UpstageEmbeddings(model="solar-embedding-1-large", embed_batch_size=100)
embeddings = embeddings_model.embed_documents(
    [
        "What is the best season to visit Korea?",
    ])

len(embeddings), len(embeddings[0])

(1, 4096)

In [70]:
# RAG 1. load doc (done), 2. chunking, splits, 3. embeding - indexing, 4. retrieve


In [71]:
from langchain_core.documents import Document

In [72]:
import json
question_and_answer_list = []
with open('json/question_and_answer_data.json', encoding='utf-8') as f:
    a = json.load(f)

In [73]:
for k, v in a.items():
    seq = ''
    for i in v:
        seq += f' {i}'
    question_and_answer_list.append(Document(page_content=seq, metadata={"test": "test"}))    

In [74]:
from langchain_chroma import Chroma

# 3. Embed & indexing
vectorstore = Chroma.from_documents(documents=question_and_answer_list, embedding=UpstageEmbeddings(model="solar-embedding-1-large", embed_batch_size=100))

챗봇 프롬프트 입력 

In [75]:
import gradio as gr

from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AIMessage, HumanMessage


llm = ChatUpstage(temperature=0.8)

In [76]:
# 4. retrive
retriever = vectorstore.as_retriever()
result_docs = retriever.invoke("전공은 어떻게 선택?")
#result_docs = retriever.invoke("{message}")
print(len(result_docs))
print(result_docs)
#print(result_docs[0].page_content[:100])

4
[Document(page_content=" Answer:  지금까지 학생의 성향을 기반으로 직무 추천을 몇 가지 해드리려고 합니다! Question:  네 Answer:  1차 추천 직업으로는 법률 및 사회활동 관련직으로, 변호사, 판사, 검사뿐 아니라 법에 관련된 일을 회사에서 하는 법무사, 저작권과 지식재산권에 대해서 다루는 변리사 등이 있어요! 요즘 저작권 부분이 굉장히 많이 다뤄지고 있기 때문에 여기도 미래 전망이 좋은 직종이랍니다 Question:  그렇군요! Answer:  방금 언급 드린 직종에 있어서 관심 있게 찾아본 게 있을까요? Question:  저는 법무사요 시험을 1, 2차로 모든 법 과목을 보는 거 같더라고요 Answer:  자세히 알아보았군요! 법무사, 변리사, 변호사의 경우 자격을 취득 후 대기업에서 근무하는 경우도 있으므로 꾸준히 법률과 관련되어 활동해보고 모의재판 동아리 등도 활동해보면 좋아요~ 법률 관련한 활동이나 사법기관 방문 같은 걸 해본 적 있을까요? Question:  법원에 한 번 가본 적 있어요 Answer:  그렇군요!  법원에 다녀오고 어떤 부분을 느끼고 왔는지 알 수 있을까요? Question:  세상에는 범죄가 끊이질 않는다는 것을 느꼈어요 Answer:  맞아요. 모든 직업에는 장단점이 존재하는데 특히 이 직무는 연봉이 높은 만큼 사실 그만큼의 정신력, 체력소모가 크고 스트레스가 많으며 업무시간, 업무량이 많은 편이에요.  중요한 사건을 다루기에 늘 긴장되고 마음이 무거울 수 있으므로 직업의 장단점을 잘 알아보기를 권해드립니다! Question:  네 Answer:  https://www.youtube.com/watch?v=wMWPkz0YBL8 유튜브에 워크맨이라는 채널 아시나요? Question:  네 Answer:  이 영상도 세상에 있는 다양한 직업을 직접 체험해보고 현직자들의 고충도 들어보는 내용이 아주 담겨있으니 쉽게 간접경험을 할 수 있는 매체로 유퀴즈와 함께 추천해 드릴게요! Questio

In [77]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage(temperature=0.5)

prompt_template = PromptTemplate.from_template(
    """
    This is the Career Counseling Guidelines
    1. Familiarize yourself with the basic history of students' individual career paths.
    2. Determine the level and type of assistance based on students' career issues.
    3. Provide students with useful career-related information and resources.
    4. Share specific examples or experiences related to career paths honestly.
    5. Tell or connect students with specific examples from their seniors' career paths.
    6. Provide detailed guidance on career exploration possibilities related to their major.
    This is the list you should not do. 
    1. Do not convey stereotypes or biases about specific careers or jobs.
    2. Do not assume that your own success experiences apply to all students.
    3. Be careful not to unilaterally guide students in the direction you want.
    4. Do not do other work during the counseling session.
    5. Do not view career counseling as a performance metric.
    6. Avoid giving advice unrelated to the students' help requests.
    7. Be cautious of providing outdated information or advice about recent job market trends and conditions.
    8. Avoid giving unilateral instructions or reprimands.
    9. Do not ask many personal questions unrelated to career counseling.
    10. Avoid saying things that might hurt or lower students' morale.
    
    Follow above rules, and find similar question in the Context and give the answer similar to corresponding Answer. 
    Suppose you are a very good student counseler. Speak like you are a good counseler.

    This is example conversation.
    "Answer: Do you have any plans for this vacation?"
    "Question: Nothing much, just exercising at home."
    "Answer: Then, before our next session, how about researching one or more things you want to do? You can use the site I mentioned or search freely."
    "Question: Sounds good."
    "Answer: The second semester of your second year will be quite an important time. It would be good to get to know yourself better. I see a cat picture in your profile. Do you like animals? Jobs related to pets are also quite promising."
    "Question: I like cats."

    You should speak the "Answer" Part. 


    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {message}
    ---
    Context: {Context}
   
    """
)
chain = prompt_template | llm | StrOutputParser()

In [78]:
def chat(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    #return chain.invoke({"message": message, "history": history_langchain_format})
    return chain.invoke({"message": message, "Context": result_docs})

In [85]:
with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(
        chat,
        examples=[
            "전공 선택은 어떻게 하는 것이 좋을까 ? ",
            "유망 직업은 뭘까?",
            "연예인이 되려면 어떻게 할까요?",
        ],
        title="Solar Chatbot",
        description="Upstage Solar Chatbot",
    )
    chatbot.chatbot.height = 300

In [86]:
chain = prompt_template | llm | StrOutputParser()

In [87]:
chain.invoke({"message":"저는 과학이 좋아요","Context":result_docs})

'질문: 저는 과학이 좋아요.'

In [89]:
if __name__ == "__main__":
    demo.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.


In [20]:
chain.invoke({"question": "저는 게임을 좋아해요.", "Context": result_docs})

'학생님께서 게임을 좋아하시는군요. 게임을 좋아하고 잘하시는 것은 멋진 일입니다. 게임을 취미로 즐기는 것도 좋지만, 프로게이머로 진로를 생각하고 계시다면, 프로게이머가 되기 위해 필요한 기술과 지식을 습득하는 것이 중요합니다.\n\n프로게이머가 되기 위해서는 게임 실력과 함께 전략적인 사고와 빠른 판단력, 그리고 팀워크 등이 필요합니다. 또한, 게임에 대한 지식과 최신 트렌드를 파악하는 것도 중요합니다.\n\n프로게이머로 진로를 선택하신다면, 게임 대회에 참가하거나 게임 관련 커뮤니티에 참여하여 경험을 쌓는 것도 좋은 방법입니다. 또한, 게임 관련 교육 기관이나 학원을 찾아보는 것도 도움이 될 수 있습니다.\n\n게임을 좋아하고 잘하시는 학생님께서는 프로게이머로 진로를 선택하신다면, 많은 가능성을 가지고 계실 것입니다. 하지만, 게임 외에도 다양한 진로가 있으니, 다른 분야에도 관심을 가지고 탐색해보시는 것도 좋은 방법입니다.\n\n학생님의 진로를 응원하며, 언제든지 도움이 필요하시면 말씀해주세요.'

# Excercise: Hybrid
Sometimes keyword search can be useful. Design a system that does keyword and semantic search, then combine the results. Use them as context for your RAG.